# Tutorial 1.5: Prompt Management

![](images/6_Prompt-Management.png)

## Version Control for Prompts

Welcome to prompt management! Prompts are the instructions that guide LLMs, and managing them properly is crucial for maintaining quality and collaboration in GenAI applications.

### What You'll Learn
- Why prompt management matters
- Creating reusable prompt templates
- Versioning prompts systematically
- Linking prompts to experiments
- Best practices for prompt organization
- Sharing prompts across teams

### Prerequisites
- Completed previous notebooks (1.1-1.4)
- Understanding of experiment tracking

### Estimated Time: 15-20 minutes

---
## Step 1: Why Manage Prompts?

### The Problem

Without proper management:

```python
# Scattered prompts in code
prompt1 = "You are a helpful assistant. Answer: {question}"
prompt2 = "You are helpful. Respond to: {question}"  # Which one works better?
prompt3 = "Answer {question}"  # Lost track of what works
```

**Problems:**
- ❌ Prompts hardcoded in multiple places
- ❌ No version history
- ❌ Hard to A/B test different versions
- ❌ Difficult to collaborate
- ❌ Can't track which prompt generated which output

### The Solution: Prompt Management

```python
# Centralized, versioned prompts
prompt_template = mlflow.log_text(template, "prompts/qa_v1.txt")
# Now you can:
# ✅ Version prompts
# ✅ Track performance
# ✅ Share with team
# ✅ Link to experiments
```

### Benefits

1. **Reproducibility**: Know exactly which prompt was used
2. **Collaboration**: Share prompts across team
3. **Experimentation**: Systematic A/B testing
4. **Version Control**: Track prompt evolution
5. **Governance**: Audit and approval processes

---
## Step 2: Environment Setup

In [1]:
import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
else:
    # Initialize OpenAI
    client = get_openai_client()
    model_name = "gpt-5-2"

print("✅ Environment configured: using", "Databricks AI Gateway" if use_databricks_provider else "OpenAI", "client")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")

# Create experiment
mlflow.set_experiment("08-prompt-management")


print("✅ Environment configured")
print("   OpenAI autologging: ENABLED")


print("✅ Environment configured for prompt management")

2026/02/04 21:10:55 INFO mlflow.tracking.fluent: Experiment with name '08-prompt-management' does not exist. Creating a new experiment.


✅ Environment configured: using Databricks AI Gateway client
   Tracking URI: http://localhost:5000
✅ Environment configured
   OpenAI autologging: ENABLED
✅ Environment configured for prompt management


---
## Step 3: Basic Prompt Logging

Let's start by logging prompts as artifacts in experiments.

In [2]:
# Basic prompt logging
with mlflow.start_run(run_name="basic-prompt-logging") as run:
    
    # Define prompt
    prompt_template = """You are a helpful AI assistant.

Answer the following question concisely:
{question}

Answer:"""
    
    # Log prompt as artifact
    mlflow.log_text(prompt_template, "prompt_template.txt")
    
    # Log prompt metadata
    mlflow.log_param("prompt_version", "v1.0")
    mlflow.log_param("prompt_author", "jules")
    mlflow.log_param("prompt_purpose", "general_qa")
    
    # Use the prompt
    question = "What is prompt engineering?"
    filled_prompt = prompt_template.format(question=question)
    
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": filled_prompt}],
        temperature=1.0
    )
    
    answer = response.choices[0].message.content
    
    # Log the filled prompt and response
    mlflow.log_text(filled_prompt, "filled_prompt.txt")
    mlflow.log_text(answer, "response.txt")
    
    print(f"Question: {question}")
    print(f"\nAnswer: {answer}")
    print("\n✅ Prompt and response logged!")
    print(f"   Run ID: {run.info.run_id}")

Question: What is prompt engineering?

Answer: Prompt engineering is the practice of designing and refining the instructions (prompts) given to an AI model to reliably produce the desired output, often by specifying context, constraints, examples, and formatting.

✅ Prompt and response logged!
   Run ID: ffe1585e4e1b4f51b38333e569ec5f97
🏃 View run basic-prompt-logging at: http://localhost:5000/#/experiments/8/runs/ffe1585e4e1b4f51b38333e569ec5f97
🧪 View experiment at: http://localhost:5000/#/experiments/8


### 💡 What We Logged

1. **Template**: The reusable prompt structure
2. **Metadata**: Version, author, purpose
3. **Filled prompt**: The actual prompt sent to LLM
4. **Response**: The LLM output

This creates a complete audit trail!

---
## Step 4: Prompt Templates with Variables

Create flexible prompt templates with multiple variables.

In [4]:
from typing import Dict, List
# Advanced prompt template
class PromptTemplate:
    """
    Simple prompt template manager.
    """
    def __init__(self, template: str, variables: List[str], metadata: Dict = None):
        self.template = template
        self.variables = variables
        self.metadata = metadata or {}
        self.metadata['created_at'] = datetime.now().isoformat()
    
    def format(self, **kwargs) -> str:
        """
        Fill template with variables.
        """
        # Validate all required variables are provided
        missing = set(self.variables) - set(kwargs.keys())
        if missing:
            raise ValueError(f"Missing variables: {missing}")
        
        return self.template.format(**kwargs)
    
    def to_dict(self) -> Dict:
        """
        Serialize to dictionary.
        """
        return {
            "template": self.template,
            "variables": self.variables,
            "metadata": self.metadata
        }
    
    def save(self, run) -> str:
        """
        Save prompt to MLflow run.
        """
        # Save template
        mlflow.log_text(self.template, "prompt_template.txt")
        
        # Save full spec
        mlflow.log_dict(self.to_dict(), "prompt_spec.json")
        
        # Log metadata as params
        for key, value in self.metadata.items():
            mlflow.log_param(f"prompt_{key}", value)
        
        return run.info.run_id

print("✅ PromptTemplate class defined")

✅ PromptTemplate class defined


### Use our generic class template, using multiple variables

In [7]:
from datetime import datetime
# Create and use a prompt template

with mlflow.start_run(run_name="template-with-variables") as run:
    
    # Define template
    template = PromptTemplate(
        template="""You are a {role}.

                 Context: {context}

                Task: {task}

                 Provide your response:""",
        variables=["role", "context", "task"],
        metadata={
            "version": "v2.0",
            "author": "jules",
            "purpose": "role_based_qa",
            "approved": True
        }
    )
    
    # Save template
    template.save(run)
    
    # Use template
    prompt = template.format(
        role="technical documentation expert",
        context="MLflow is an open source MLOps platform",
        task="Explain MLflow tracing in 2 sentences"
    )
    
    # Make LLM call
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    
    answer = response.choices[0].message.content
    
    # Log filled prompt and response
    mlflow.log_text(prompt, "filled_prompt.txt")
    mlflow.log_text(answer, "response.txt")
    
    print(f"Answer:\n{answer}")
    print("\n✅ Template-based prompt logged!")
    print("\n📋 Template info:")
    print(f"   Variables: {template.variables}")
    print(f"   Version: {template.metadata['version']}")

Answer:
MLflow Tracing captures detailed, end-to-end execution traces of your ML or LLM application—recording spans for steps like retrieval, model calls, and tool/function invocations along with timing, inputs/outputs, and metadata. These traces are logged to MLflow so you can visualize and debug runs, identify bottlenecks, and monitor quality and reliability in production.

✅ Template-based prompt logged!

📋 Template info:
   Variables: ['role', 'context', 'task']
   Version: v2.0
🏃 View run template-with-variables at: http://localhost:5000/#/experiments/8/runs/f572c01bdc314d368968a03a52720676
🧪 View experiment at: http://localhost:5000/#/experiments/8


---
## Step 5: Prompt Versioning

Track prompt evolution over time, with more specificity such as role, question, answer, and guidlines.

In [8]:
# Version 1: Basic prompt, no role
prompts = {
    "v1.0": {
        "template": "Answer this question: {question}",
        "changes": "Initial version"
    },
    "v1.1": {
        "template": "You are helpful. Answer concisely: {question}",
        "changes": "Added role and instruction for conciseness"
    },
# version 2: with role, question and answer
    "v2.0": {
        "template": """You are a helpful AI assistant. Answer the question concisely and accurately.
        Question: {question}
        Answer:""",
        "changes": "Structured format, emphasized accuracy"
    },
# version 3: with role, question and answer, and guidelines
    "v3.0": {
        "template": """You are a helpful AI assistant specializing in technical topics.

        Guidelines:
            - Answer concisely (2-3 sentences max)
            - Be accurate and factual
            - If unsure, say so

        Question: {question}

        Answer:""",
        "changes": "Added specialization, explicit guidelines"
    }
}

# Test all versions
question = "What is MLflow?"
results = []

print("\n🔄 Testing prompt versions...\n")

for version, prompt_info in prompts.items():
    with mlflow.start_run(run_name=f"prompt_{version}") as run:
        
        # Log prompt metadata
        mlflow.log_param("prompt_version", version)
        mlflow.log_param("changes", prompt_info["changes"])
        
        # Log template
        mlflow.log_text(prompt_info["template"], "prompt_template.txt")
        
        # Fill and use template
        filled_prompt = prompt_info["template"].format(question=question)
        
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": filled_prompt}],
            temperature=1.0
        )
        
        answer = response.choices[0].message.content
        tokens = response.usage.total_tokens
        
        # Log results
        mlflow.log_text(answer, "response.txt")
        mlflow.log_metric("response_length", len(answer))
        mlflow.log_metric("total_tokens", tokens)
        
        results.append({
            "version": version,
            "answer": answer,
            "length": len(answer),
            "tokens": tokens
        })
        
        print(f"{version}: {answer[:60]}... ({len(answer)} chars)")

print("\n✅ All versions tested and logged!")
print("\n📊 Compare in MLflow UI:")
print("   - Select all runs")
print("   - Compare responses")
print("   - Analyze metrics (length, tokens)")
print("   - Choose the best version!")


🔄 Testing prompt versions...

v1.0: MLflow is an open-source platform for managing the end-to-en... (705 chars)
🏃 View run prompt_v1.0 at: http://localhost:5000/#/experiments/8/runs/2e1eded2b36d4d2db7029931c54beded
🧪 View experiment at: http://localhost:5000/#/experiments/8
v1.1: MLflow is an open-source platform for managing the end-to-en... (195 chars)
🏃 View run prompt_v1.1 at: http://localhost:5000/#/experiments/8/runs/5d8eb55c70d04aa182c12bbf9c5cfc92
🧪 View experiment at: http://localhost:5000/#/experiments/8
v2.0: MLflow is an open-source platform for managing the end-to-en... (291 chars)
🏃 View run prompt_v2.0 at: http://localhost:5000/#/experiments/8/runs/778171a6817a451b8c6222cafcf9bd22
🧪 View experiment at: http://localhost:5000/#/experiments/8
v3.0: MLflow is an open-source platform for managing the end-to-en... (337 chars)
🏃 View run prompt_v3.0 at: http://localhost:5000/#/experiments/8/runs/84252b62ee154a9387e072bc83ec878f
🧪 View experiment at: http://localhost:5000/#/exp

### 📈 Prompt Version Analysis

By versioning prompts, you can:

1. **Track evolution**: See how prompts improve over time
2. **A/B test**: Compare versions systematically
3. **Rollback**: Revert to previous version if needed
4. **Document changes**: Know why each change was made
5. **Measure impact**: Did the change improve quality?

**All visible in MLflow experiments!**

---
## Step 6: Prompt Library

Create a reusable prompt library for your organization, a practice similar to create resuable or shareable code library

In [10]:
# Prompt library
PROMPT_LIBRARY = {
    "qa_simple": {
        "template": "Answer this question: {question}",
        "variables": ["question"],
        "use_case": "Simple Q&A",
        "recommended_temp": 1.0
    },
    "qa_with_context": {
        "template": """Use the following context to answer the question.
Context: {context}

Question: {question}

Answer based only on the context above:""",
        "variables": ["context", "question"],
        "use_case": "RAG Q&A",
        "recommended_temp": 0.1
    },
    "summarization": {
        "template": """Summarize the following text in {length} sentences.

Text: {text}

Summary:""",
        "variables": ["text", "length"],
        "use_case": "Text summarization",
        "recommended_temp": 0.5
    },
    "classification": {
        "template": """Classify the following text into one of these categories: {categories}

Text: {text}

Category:""",
        "variables": ["text", "categories"],
        "use_case": "Text classification",
        "recommended_temp": 0.0
    },
    "creative_writing": {
        "template": """Write a creative {format} about {topic}.

Style: {style}

Your {format}:""",
        "variables": ["format", "topic", "style"],
        "use_case": "Creative content generation",
        "recommended_temp": 0.9
    }
}

# Save prompt library
with mlflow.start_run(run_name="prompt-library") as run:
    
    mlflow.log_dict(PROMPT_LIBRARY, "prompt_library.json")
    mlflow.set_tag("artifact_type", "prompt_library")
    mlflow.set_tag("num_prompts", len(PROMPT_LIBRARY))
    
    print("✅ Prompt library saved!")
    print("\n📚 Available prompts:")
    for name, prompt in PROMPT_LIBRARY.items():
        print(f"   - {name}: {prompt['use_case']}")
    
    print(f"\n🔗 Library saved to run: {run.info.run_id}")

✅ Prompt library saved!

📚 Available prompts:
   - qa_simple: Simple Q&A
   - qa_with_context: RAG Q&A
   - summarization: Text summarization
   - classification: Text classification
   - creative_writing: Creative content generation

🔗 Library saved to run: 6c719c8269ae4dab83ac79c9653f078b
🏃 View run prompt-library at: http://localhost:5000/#/experiments/8/runs/6c719c8269ae4dab83ac79c9653f078b
🧪 View experiment at: http://localhost:5000/#/experiments/8


In [11]:
# Use a prompt from the library
def use_library_prompt(prompt_name: str, **kwargs):
    """
    Load and use a prompt from the library.
    """
    if prompt_name not in PROMPT_LIBRARY:
        raise ValueError(f"Prompt '{prompt_name}' not found in library")
    
    prompt_info = PROMPT_LIBRARY[prompt_name]
    
    with mlflow.start_run(run_name=f"using_{prompt_name}"):
        
        # Log which prompt we're using
        mlflow.log_param("prompt_name", prompt_name)
        mlflow.log_param("use_case", prompt_info["use_case"])
        mlflow.log_param("recommended_temp", prompt_info["recommended_temp"])
        
        # Fill template
        prompt = prompt_info["template"].format(**kwargs)
        mlflow.log_text(prompt, "filled_prompt.txt")
        
        # Make LLM call
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=prompt_info["recommended_temp"]
        )
        
        answer = response.choices[0].message.content
        mlflow.log_text(answer, "response.txt")
        
        return answer

# Test the library
print("\n📚 Testing prompt library...\n")

# Example 1: RAG Q&A
answer1 = use_library_prompt(
    "qa_with_context",
    context="MLflow Tracing provides observability for GenAI applications.",
    question="What does MLflow Tracing do?"
)
print(f"RAG Q&A: {answer1}\n")

# Example 2: Classification
answer2 = use_library_prompt(
    "classification",
    text="I love using MLflow for my ML projects!",
    categories="Positive, Negative, Neutral"
)
print(f"Classification: {answer2}")

print("\n✅ Prompt library works!")


📚 Testing prompt library...

🏃 View run using_qa_with_context at: http://localhost:5000/#/experiments/8/runs/09bc974c368146899941688813c1c34b
🧪 View experiment at: http://localhost:5000/#/experiments/8
RAG Q&A: MLflow Tracing provides observability for GenAI applications.

🏃 View run using_classification at: http://localhost:5000/#/experiments/8/runs/55954aa8160a4dbe8b228bdf1d702990
🧪 View experiment at: http://localhost:5000/#/experiments/8
Classification: Positive

✅ Prompt library works!


### 📚 Prompt Library Benefits

A centralized prompt library provides:

1. **Consistency**: Everyone uses approved prompts
2. **Reusability**: Don't reinvent the wheel
3. **Best practices**: Recommended settings included
4. **Easy updates**: Change once, affect all users
5. **Governance**: Control what gets deployed

**Essential for team collaboration!**

---
## Step 7: MLflow Prompt Registry

Take your prompt management to production with **MLflow's Prompt Registry** - a centralized repository that provides versioning, aliases, and search capabilities.

These capablities and features do not exist if you manually creatred a Python Dict as we did in step 6

### Why Use the Prompt Registry?

The Prompt Registry builds on the manual approach from Step 6, adding:

| Feature | Manual (Step 6) | Prompt Registry |
|---------|-----------------|-----------------|
| Versioning | Manual params | Automatic with history |
| Aliases | Not available | Built-in (prod/staging) |
| Search | Not available | Tag-based filtering |
| UI | Artifacts tab | Dedicated Registry view |
| Rollback | Manual | One-click alias update |

Let's migrate our `PROMPT_LIBRARY` to the Prompt Registry!

In [12]:
# Register prompts from PROMPT_LIBRARY to MLflow Prompt Registry
# Note: Prompt Registry uses Jinja2 syntax {{ variable }} instead of Python {variable}

import re

def convert_to_jinja2(template: str) -> str:
    """Convert Python format {var} to Jinja2 {{ var }} syntax."""
    # Match {word} but not {{ or }}
    return re.sub(r'\{(\w+)\}', r'{{ \1 }}', template)

print("🔄 Registering prompts to MLflow Prompt Registry...\n")

registered_prompts = {}
for name, prompt_info in PROMPT_LIBRARY.items():
    prompt_name = f"tutorial-{name}"
    
    # Convert template to Jinja2 syntax
    jinja_template = convert_to_jinja2(prompt_info["template"])
    
    # Register to Prompt Registry
    prompt = mlflow.genai.register_prompt(
        name=prompt_name,
        template=jinja_template,
        commit_message=f"Initial version - {prompt_info['use_case']}",
        tags={
            "use_case": prompt_info["use_case"],
            "recommended_temp": str(prompt_info["recommended_temp"]),
            "variables": ",".join(prompt_info["variables"]),
            "author": "jules"
        }
    )
    
    registered_prompts[name] = prompt
    print(f"✅ Registered: {prompt_name} (version {prompt.version})")

print(f"\n📋 Total prompts registered: {len(registered_prompts)}")
print("   View them in MLflow UI → Prompt Registry")

2026/02/04 21:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: tutorial-qa_simple, version 1
2026/02/04 21:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: tutorial-qa_with_context, version 1
2026/02/04 21:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: tutorial-summarization, version 1
2026/02/04 21:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: tutorial-classification, version 1


🔄 Registering prompts to MLflow Prompt Registry...

✅ Registered: tutorial-qa_simple (version 1)
✅ Registered: tutorial-qa_with_context (version 1)
✅ Registered: tutorial-summarization (version 1)
✅ Registered: tutorial-classification (version 1)


2026/02/04 21:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: tutorial-creative_writing, version 1


✅ Registered: tutorial-creative_writing (version 1)

📋 Total prompts registered: 5
   View them in MLflow UI → Prompt Registry


### Loading and Using Registry Prompts

Load prompts by version or alias, then use `format()` to fill variables.

In [13]:
# Load a prompt from the registry and use it
import mlflow

mlflow.openai.autolog()

print("📥 Loading and using a prompt from the Registry...\n")

# Load by name (gets latest version)
prompt = mlflow.genai.load_prompt("tutorial-qa_with_context")

print("Loaded prompt: tutorial-qa_with_context")
print(f"   Version: {prompt.version}")
print(f"   Template preview: {prompt.template[:50]}...")

# Fill the template with variables
filled_prompt = prompt.format(
    context="MLflow Tracing provides observability for GenAI applications. It captures execution traces, latency metrics, and token usage.",
    question="What does MLflow Tracing provide?"
)

print(f"\n📝 Filled prompt:\n{filled_prompt}")

# Use the filled prompt with the LLM
response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": filled_prompt}],
    temperature=0.1
)

answer = response.choices[0].message.content
print(f"\n🤖 Answer: {answer}")

📥 Loading and using a prompt from the Registry...

Loaded prompt: tutorial-qa_with_context
   Version: 1
   Template preview: Use the following context to answer the question.
...

📝 Filled prompt:
Use the following context to answer the question.
Context: MLflow Tracing provides observability for GenAI applications. It captures execution traces, latency metrics, and token usage.

Question: What does MLflow Tracing provide?

Answer based only on the context above:

🤖 Answer: MLflow Tracing provides observability for GenAI applications by capturing execution traces, latency metrics, and token usage.


Trace(trace_id=tr-c73034a3bbba5c5a441edd4e3cb8bbfd)

### Aliases for Environment Management

Aliases let you reference prompts by environment (e.g., `production`, `staging`) instead of version numbers. This enables safe deployments and easy rollbacks.

In [14]:
#
# Set up aliases for the RAG Q&A prompt
print("🏷️ Setting up aliases for environment management...\n")

# Set version 1 as production
mlflow.genai.set_prompt_alias("tutorial-qa_with_context", "production", version=1)
print("✅ Set 'production' alias → version 1")

# Set version 1 as staging (in real use, you'd have a newer version here)
mlflow.genai.set_prompt_alias("tutorial-qa_with_context", "staging", version=1)
print("✅ Set 'staging' alias → version 1")

# Load prompt via alias - this is what production code should use!
print("\n📥 Loading prompt via alias...")
prod_prompt = mlflow.genai.load_prompt("prompts:/tutorial-qa_with_context@production")
print(f"   Loaded production prompt (version {prod_prompt.version})")

# Use the production prompt
filled = prod_prompt.format(
    context="MLflow is an open source platform for the ML lifecycle.",
    question="What is MLflow?"
)

response = client.chat.completions.create(
    model=model_name,
    messages=[{"role": "user", "content": filled}],
    temperature=0.1
)

print(f"\n🤖 Production response: {response.choices[0].message.content}")
print("\n💡 Tip: Update the alias to point to a new version for safe rollouts!")

🏷️ Setting up aliases for environment management...

✅ Set 'production' alias → version 1
✅ Set 'staging' alias → version 1

📥 Loading prompt via alias...
   Loaded production prompt (version 1)

🤖 Production response: MLflow is an open source platform for the machine learning (ML) lifecycle.

💡 Tip: Update the alias to point to a new version for safe rollouts!


Trace(trace_id=tr-4a11504cf985427d24046915622debaa)

### Versioning with Commit Messages

Register improved versions with descriptive commit messages to track changes.

In [ ]:
# Register an improved version of the RAG Q&A prompt
print("📝 Creating a new version with improvements...\n")

improved_template = """You are a helpful AI assistant. Use ONLY the provided context to answer.

Context:
{{ context }}

Question: {{ question }}

Instructions:
- Answer based strictly on the context above
- If the answer is not in the context, say "I don't have enough information"
- Be concise (2-3 sentences max)

Answer:"""

# Register as version 2
updated_prompt = mlflow.genai.register_prompt(
    name="tutorial-qa_with_context",
    template=improved_template,
    commit_message="Added instructions for stricter context adherence and conciseness",
    tags={
        "use_case": "RAG Q&A",
        "recommended_temp": "0.1",
        "variables": "context,question",
        "author": "jules"
    }
)

print(f"✅ Registered version {updated_prompt.version}")
print("   Commit: Added instructions for stricter context adherence")

# Now promote to staging for testing
mlflow.genai.set_prompt_alias("tutorial-qa_with_context", "staging", version=updated_prompt.version)
print(f"\n🏷️ Promoted version {updated_prompt.version} to 'staging' alias")
print("   Production still points to version 1 (safe rollout!)")

### Searching Prompts

Discover prompts across your organization using tag-based search.

In [15]:
# Search for prompts by tags
print("🔍 Searching prompts in the Registry...\n")

# Search all prompts by author
all_prompts = mlflow.genai.search_prompts(filter_string="tags.author='jules'")

print(f"Found {len(all_prompts)} prompts by author 'jules':")
for p in all_prompts:
    print(f"   - {p.name}")

# You can also filter by use_case, or any other tag
print("\n💡 Search tips:")
print("   - Filter by author: tags.author='jules'")
print("   - Filter by use case: tags.use_case='RAG Q&A'")
print("   - Combine filters: tags.author='jules' AND tags.use_case='RAG Q&A'")

🔍 Searching prompts in the Registry...

Found 5 prompts by author 'jules':
   - tutorial-classification
   - tutorial-creative_writing
   - tutorial-qa_simple
   - tutorial-qa_with_context
   - tutorial-summarization

💡 Search tips:
   - Filter by author: tags.author='jules'
   - Filter by use case: tags.use_case='RAG Q&A'
   - Combine filters: tags.author='jules' AND tags.use_case='RAG Q&A'


---
## Step 8: Best Practices

Let's review prompt management best practices.

In [16]:
print("""
╔══════════════════════════════════════════════════════════════╗
║         Prompt Management Best Practices                     ║
╚══════════════════════════════════════════════════════════════╝

📝 CREATION:

1. Use descriptive names
   ✅ "rag_qa_with_context_v2"
   ❌ "prompt1"

2. Document purpose and use case
   - What task is this for?
   - What model works best?
   - What temperature is recommended?

3. Define clear variables
   - Use {descriptive_names}
   - Document expected format
   - Validate inputs

4. Include examples in metadata
   - Show good inputs
   - Show expected outputs

🔄 VERSIONING:

1. Use semantic versioning
   - v1.0: Major version (breaking changes)
   - v1.1: Minor version (improvements)
   - v1.1.1: Patch (bug fixes)

2. Document changes
   - What changed?
   - Why did it change?
   - What's the expected impact?

3. Test before deploying
   - Compare with previous version
   - Measure quality metrics
   - Get stakeholder approval

📊 TRACKING:

1. Link prompts to experiments
   - Log prompt version in every run
   - Enable filtering by prompt

2. Track performance
   - Quality metrics
   - Token usage
   - Latency
   - Cost

3. Monitor in production
   - Which prompts are used most?
   - Are there quality issues?
   - When to update?

👥 COLLABORATION:

1. Centralize prompt storage
   - One source of truth
   - MLflow artifacts or dedicated system

2. Implement review process
   - Peer review new prompts
   - Test thoroughly
   - Document decisions

3. Share learnings
   - What works well?
   - What to avoid?
   - Common patterns?

🚀 DEPLOYMENT:

1. Gradual rollout
   - Test with small percentage
   - Monitor closely
   - Be ready to rollback

2. Have fallback prompts
   - If new prompt fails
   - Automatic revert?
   - Monitoring alerts?

3. Document production prompts
   - Which version is live?
   - When was it deployed?
   - What's the rollback plan?

""")


╔══════════════════════════════════════════════════════════════╗
║         Prompt Management Best Practices                     ║
╚══════════════════════════════════════════════════════════════╝

📝 CREATION:

1. Use descriptive names
   ✅ "rag_qa_with_context_v2"
   ❌ "prompt1"

2. Document purpose and use case
   - What task is this for?
   - What model works best?
   - What temperature is recommended?

3. Define clear variables
   - Use {descriptive_names}
   - Document expected format
   - Validate inputs

4. Include examples in metadata
   - Show good inputs
   - Show expected outputs

🔄 VERSIONING:

1. Use semantic versioning
   - v1.0: Major version (breaking changes)
   - v1.1: Minor version (improvements)
   - v1.1.1: Patch (bug fixes)

2. Document changes
   - What changed?
   - Why did it change?
   - What's the expected impact?

3. Test before deploying
   - Compare with previous version
   - Measure quality metrics
   - Get stakeholder approval

📊 TRACKING:

1. Link prompt

---
## Summary

In this notebook, you learned:

1. ✅ Why prompt management is crucial for GenAI applications
2. ✅ How to log prompts as artifacts in MLflow
3. ✅ Creating flexible prompt templates with variables
4. ✅ Systematic prompt versioning
5. ✅ Building a reusable prompt library
6. ✅ **Using MLflow Prompt Registry** for production prompt management
7. ✅ **Registering and versioning prompts** with commit messages
8. ✅ **Using aliases** (production/staging) for safe deployments
9. ✅ **Searching prompts** by tags for discovery
10. ✅ Best practices for team collaboration

### Key Takeaways

- **Treat prompts as code**: Version, test, review
- **Use Prompt Registry**: Centralized, versioned, searchable
- **Leverage aliases**: Safe deployments with production/staging
- **Track everything**: Link prompts to experiments
- **Iterate systematically**: A/B test versions
- **Document thoroughly**: Purpose, use case, commit messages

### What's Next?

**📓 Notebook 1.6: Framework Integrations**

Learn how to:
- Work with different GenAI frameworks
- Compare OpenAI, LangChain, LlamaIndex
- Choose the right tool for your use case
- Integrate MLflow with your framework
- Best practices for each framework